Import the "CSVdownaload.csv" that is keyed by county,date and convert it into a frequency table.<br> 
Note: CSVdownlaod.csv for 2020-10-14 renamed to tests_1014

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
#Read from File
#df = pd.read_csv("CSVDownload.csv",parse_dates=True)

#Or copy from the web
import requests
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
urlPath = "EpiCurveApr4/CSVDownload.csv"
r = requests.get(
    f"https://public.tableau.com/views/{urlPath}",
    params= {
        ":showVizHome":"no",
    }
)
with StringIO(r.text) as f:
    df = pd.read_csv(f)


In [3]:
df.dtypes

County                   object
Date                     object
New Cases                 int64
New Positive Tests        int64
Total Test Encounters     int64
dtype: object

In [4]:
df["Date"] = pd.to_datetime(df["Date"],infer_datetime_format=True)

In [5]:
df.dtypes

County                           object
Date                     datetime64[ns]
New Cases                         int64
New Positive Tests                int64
Total Test Encounters             int64
dtype: object

In [6]:
df.columns

Index(['County', 'Date', 'New Cases', 'New Positive Tests',
       'Total Test Encounters'],
      dtype='object')

In [7]:
df = df.rename(columns={'New Cases':'Cases','New Positive Tests':'Pos','Total Test Encounters':'Tests'},errors='raise')

In [8]:
df.columns

Index(['County', 'Date', 'Cases', 'Pos', 'Tests'], dtype='object')

In [9]:
df['County'].unique()

array(['Hawaii', 'Honolulu', 'Kauai', 'Maui', 'Missing'], dtype=object)

In [10]:
dfHawaii = df.loc[df['County']=='Hawaii',['Date','Cases','Pos','Tests']]
dfOahu = df.loc[df['County']=='Honolulu',['Date','Cases','Pos','Tests']]
dfKauai = df.loc[df['County']=='Kauai',['Date','Cases','Pos','Tests']]
dfMaui = df.loc[df['County']=='Maui',['Date','Cases','Pos','Tests']]
dfMissing = df.loc[df['County']=='Missing',['Date','Cases','Pos','Tests']]

In [11]:
dfHawaii.set_index('Date',inplace=True)
dfOahu.set_index('Date',inplace=True)
dfKauai.set_index('Date',inplace=True)
dfMaui.set_index('Date',inplace=True)
dfMissing.set_index('Date',inplace=True)

In [12]:
dfHawaii = dfHawaii.rename(columns={'Cases':'Hawaii Cases','Pos':'Hawaii Pos','Tests':'Hawaii Tests'})
dfOahu = dfOahu.rename(columns={'Cases':'Oahu Cases','Pos':'Oahu Pos','Tests':'Oahu Tests'})
dfKauai = dfKauai.rename(columns={'Cases':'Kauai Cases','Pos':'Kauai Pos','Tests':'Kauai Tests'})
dfMaui = dfMaui.rename(columns={'Cases':'Maui Cases','Pos':'Maui Pos','Tests':'Maui Tests'})
dfMissing = dfMissing.rename(columns={'Cases':'Missing Cases','Pos':'Missing Pos','Tests':'Missing Tests'})

In [13]:
dfHawaii = dfHawaii.asfreq('D',fill_value=0)
dfOahu = dfOahu.asfreq('D',fill_value=0)
dfKauai = dfKauai.asfreq('D',fill_value=0)
dfMaui = dfMaui.asfreq('D',fill_value=0)
dfMissing = dfMissing.asfreq('D',fill_value=0)

In [14]:
df = dfOahu.join(dfHawaii)
df = df.join(dfMaui)
df = df.join(dfKauai)
df = df.join(dfMissing)

In [15]:
df = df.fillna(0)

In [16]:
df.to_csv('tests.csv')

In [17]:
(df['Oahu Cases'].tail(7).mean() / df['Oahu Tests'].tail(7).mean()) * 100

2.930032836574893

In [18]:
df[['Oahu Tests','Missing Tests']].tail(7)

,Oahu Tests,Missing Tests
Date,,
2020-10-13,2766,178
2020-10-14,2364,256
2020-10-15,2303,118
2020-10-16,2767,272
2020-10-17,2192,200
2020-10-18,1035,61
2020-10-19,2409,146


In [19]:
(df['Missing Tests'].tail(7) / 
df[['Oahu Tests','Hawaii Tests','Maui Tests','Kauai Tests','Missing Tests']].tail(7).sum(axis=1))*100

Date
2020-10-13    4.430065
2020-10-14    7.174888
2020-10-15    3.091433
2020-10-16    6.822172
2020-10-17    5.434783
2020-10-18    3.690260
2020-10-19    4.271504
Freq: D, dtype: float64

In [20]:
df['Missing Tests'].tail(7) / df['Oahu Tests'].tail(7)

Date
2020-10-13    0.064353
2020-10-14    0.108291
2020-10-15    0.051238
2020-10-16    0.098301
2020-10-17    0.091241
2020-10-18    0.058937
2020-10-19    0.060606
Freq: D, dtype: float64

In [21]:
((df['Oahu Cases'] / df[['Oahu Tests','Missing Tests']].sum(axis=1))*100).rolling(7).mean().tail(21)

Date
2020-09-29    3.390466
2020-09-30    3.488340
2020-10-01    3.290031
2020-10-02    3.118427
2020-10-03    3.020066
2020-10-04    2.854101
2020-10-05    2.804696
2020-10-06    2.896847
2020-10-07    2.803547
2020-10-08    3.060749
2020-10-09    3.080501
2020-10-10    3.233263
2020-10-11    3.368491
2020-10-12    3.256679
2020-10-13    3.146539
2020-10-14    3.079492
2020-10-15    2.864768
2020-10-16    2.887772
2020-10-17    2.848341
2020-10-18    2.625395
2020-10-19    2.730855
Freq: D, dtype: float64

In [22]:
dfMissing.describe()

,Missing Cases,Missing Pos,Missing Tests
count,235.0,235.0,235.000000
mean,0.0,0.0,125.238298
std,0.0,0.0,119.991821
min,0.0,0.0,0.000000
25%,0.0,0.0,41.500000
50%,0.0,0.0,77.000000
75%,0.0,0.0,177.000000
max,0.0,0.0,553.000000


In [23]:
dfOahu.describe()

,Oahu Cases,Oahu Pos,Oahu Tests
count,235.000000,235.000000,235.000000
mean,53.412766,53.285106,1430.727660
std,73.495129,72.890560,1541.847865
min,0.000000,0.000000,0.000000
25%,2.000000,2.000000,398.000000
50%,17.000000,17.000000,916.000000
75%,86.000000,87.500000,2193.000000
max,342.000000,333.000000,9184.000000


In [24]:
df.loc[df['Oahu Tests']==0]

,Oahu Cases,Oahu Pos,Oahu Tests,Hawaii Cases,Hawaii Pos,Hawaii Tests,Maui Cases,Maui Pos,Maui Tests,Kauai Cases,Kauai Pos,Kauai Tests,Missing Cases,Missing Pos,Missing Tests
Date,,,,,,,,,,,,,,,
2020-02-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2020-02-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-05,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [25]:
dfOahu.tail(7)

,Oahu Cases,Oahu Pos,Oahu Tests
Date,,,
2020-10-13,80,81,2766
2020-10-14,74,74,2364
2020-10-15,69,78,2303
2020-10-16,81,75,2767
2020-10-17,68,74,2192
2020-10-18,31,27,1035
2020-10-19,61,56,2409


In [26]:
df.shift(1,'D')

,Oahu Cases,Oahu Pos,Oahu Tests,Hawaii Cases,Hawaii Pos,Hawaii Tests,Maui Cases,Maui Pos,Maui Tests,Kauai Cases,Kauai Pos,Kauai Tests,Missing Cases,Missing Pos,Missing Tests
Date,,,,,,,,,,,,,,,
2020-02-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2020-03-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-03,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-04,0,0,2,0,0,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-16,69,78,2303,20,21,1061,0,0,257,0,0,78,0,0,118
2020-10-17,81,75,2767,12,13,525,0,0,313,1,1,110,0,0,272
2020-10-18,68,74,2192,14,17,797,0,0,359,0,1,132,0,0,200


In [27]:
dfManual = pd.read_csv("../data-kDl1L.csv",parse_dates=True)

In [28]:
dfManual["Date"] = pd.to_datetime(dfManual["Date"],infer_datetime_format=True)

In [29]:
dfManual.set_index("Date",inplace=True)

In [30]:
dfManual.columns

Index(['Statewide total', 'Oahu', 'Hawaii island', 'Maui', 'Kauai', 'Molokai',
       'Outside Hawaii'],
      dtype='object')

In [31]:
dfTest = df[['Oahu Cases','Hawaii Cases','Maui Cases','Kauai Cases']].shift(1,'D')

In [32]:
dfTest['Total'] = dfTest.sum(axis=1)

In [33]:
dfTest = dfTest.join(dfManual)

In [34]:
(dfTest['Total']-dfTest['Statewide total']).describe()

count    229.000000
mean      -0.222707
std        0.605682
min       -4.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.000000
dtype: float64

In [35]:
dfTest['Total'] =dfTest['Total'] + dfTest['Outside Hawaii']

In [36]:
dfTest.tail(21)

,Oahu Cases,Hawaii Cases,Maui Cases,Kauai Cases,Total,Statewide total,Oahu,Hawaii island,Maui,Kauai,Molokai,Outside Hawaii
Date,,,,,,,,,,,,
2020-09-30,94,26,1,0,122.0,122.0,94.0,26.0,1.0,0.0,0.0,1.0
2020-10-01,101,6,0,0,107.0,107.0,101.0,6.0,0.0,0.0,0.0,0.0
2020-10-02,69,16,0,0,86.0,86.0,69.0,16.0,0.0,0.0,0.0,1.0
2020-10-03,87,43,3,0,133.0,133.0,87.0,43.0,3.0,0.0,0.0,0.0
2020-10-04,53,15,1,0,69.0,69.0,53.0,15.0,1.0,0.0,0.0,0.0
2020-10-05,41,10,0,0,51.0,51.0,41.0,10.0,0.0,0.0,0.0,0.0
2020-10-06,67,13,1,0,81.0,81.0,67.0,13.0,1.0,0.0,0.0,0.0
2020-10-07,90,18,1,0,111.0,111.0,90.0,18.0,1.0,0.0,0.0,2.0
2020-10-08,86,14,1,0,101.0,101.0,86.0,14.0,1.0,0.0,0.0,0.0


In [37]:
dfTest['Oahu Cases'].tail(21).sum()-dfTest['Oahu'].tail(21).sum()

-1.0

In [38]:
dfManual.columns

Index(['Statewide total', 'Oahu', 'Hawaii island', 'Maui', 'Kauai', 'Molokai',
       'Outside Hawaii'],
      dtype='object')

In [39]:
dfOut = dfTest[['Total','Oahu Cases','Hawaii Cases','Maui Cases','Kauai Cases','Molokai','Outside Hawaii']]

In [40]:
dfOut = dfOut.rename(columns={'Total':'Statewide total','Oahu Cases':'Oahu',
                              'Hawaii Cases':'Hawaii island','Maui Cases':'Maui','Kauai Cases':'Kauai'})

In [41]:
dfOut.tail(14)

,Statewide total,Oahu,Hawaii island,Maui,Kauai,Molokai,Outside Hawaii
Date,,,,,,,
2020-10-07,111.0,90,18,1,0,0.0,2.0
2020-10-08,101.0,86,14,1,0,0.0,0.0
2020-10-09,152.0,107,44,0,0,0.0,1.0
2020-10-10,73.0,58,15,0,0,0.0,0.0
2020-10-11,103.0,79,24,0,0,0.0,0.0
2020-10-12,42.0,37,5,0,0,0.0,0.0
2020-10-13,62.0,30,21,11,0,0.0,0.0
2020-10-14,100.0,80,18,1,0,0.0,1.0
2020-10-15,90.0,74,13,1,0,0.0,2.0


In [42]:
dfOut.to_csv("data-kDl1L.csv",float_format='%.0f',date_format='%m/%d/%Y')